In [9]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

from funciones_varias import cargaCsvToDataFrame

dfMovies = cargaCsvToDataFrame("dfMovies", "datasets_eda")
dfCrew = cargaCsvToDataFrame("dfCrew", "datasets_eda")
dfCast = cargaCsvToDataFrame("dfCast", "datasets_eda")

ImportError: cannot import name 'cambiarDataType' from 'funciones_varias' (c:\Users\Administrator\Desktop\henry proyectos\peliculas_recomendacion_MLOP\funciones_varias.py)

In [6]:
#considero que para el modelo de clasificacion no necesito estas columnas
dfMovies = dfMovies.drop(columns=["tagline", "id", "status"])

In [25]:
dfMovies
#movie_title string
#movie_genre string
#movie_genre string
#original_language string
#overview string
#production_company string
#language string

#release_date date

#vote_count int
#runtime int
#release_year int

#return float
#budget float
#vote_average float
#revenue float
#popularity float

strings= ["movie_title", "movie_genre", "original_language", "overview", "production_company", "language"]
integers=["vote_count", "runtime", "release_year"]




#para revisar valores raros en una columna
counter=0
for i in dfMovies["vote_count"]:
    if i == "":
        print(i)
        counter+=1
print(counter)

dfMovies["vote_count"].replace("", 0, inplace=True) #hay strings vacios en vote_count por eos los saco

for i in strings:
    dfMovies[i] = dfMovies[i].astype("string")
for i in integers:
    dfMovies[i] = dfMovies[i].astype("int")


dfMovies.dtypes

0


ValueError: invalid literal for int() with base 10: '81.0'

In [4]:
#dropeo valor raro porque sino no puedo ponerle el data type que corresponde
dfMovies.drop(dfMovies[dfMovies['popularity'] == 'Beware Of Frost Bites'].index, inplace=True)

dfMovies.replace("", 0, inplace=True)
dfMovies["budget"] = dfMovies["budget"].astype(float)
dfMovies["popularity"] = dfMovies["popularity"].astype(float) #tiene valor muy raro: 'Beware Of Frost Bites'
dfMovies["revenue"] = dfMovies["revenue"].astype(float)
dfMovies["runtime"] = dfMovies["runtime"].astype(float)
dfMovies["vote_average"] = dfMovies["vote_average"].astype(float)
dfMovies["return"] = dfMovies["return"].astype(float)

,,movie_title,budget,movie_genre,franquicia,id,original_language,overview,popularity,production_company,...,release_date,release_year,revenue,runtime,language,status,tagline,vote_average,vote_count,return
0,0,Toy Story,30000000.0,Animation,Toy Story Collection,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,...,1995-10-30,1995,373554033.0,81.0,English,Released,,7.7,5415,12.45
1,1,Jumanji,65000000.0,Adventure,,8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,...,1995-12-15,1995,262797249.0,104.0,English,Released,Roll the dice and unleash the excitement!,6.9,2413,4.04
2,2,Grumpier Old Men,0.0,Romance,Grumpy Old Men Collection,15602,en,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.,...,1995-12-22,1995,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,6.5,92,0.0
3,3,Waiting to Exhale,16000000.0,Comedy,,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,...,1995-12-22,1995,81452156.0,127.0,English,Released,Friends are the people who let you be yourself...,6.1,34,5.09
4,4,Father of the Bride Part II,0.0,Comedy,Father of the Bride Collection,11862,en,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions,...,1995-02-10,1995,76578911.0,106.0,English,Released,Just When His World Is Back To Normal... He's ...,5.7,173,0.0
